In [99]:
import numpy as np
import pandas as pd
import os
import sklearn
from sklearn import metrics

In [100]:
# Data source: https://www.kaggle.com/datasets/gauthamp10/google-playstore-apps
path = os.getcwd() + "/data/Google-Playstore.csv"
df = pd.read_csv(path)

In [101]:
df = df.drop(['Scraped Time', 'Developer Id', 'Installs', 'Currency', 'Minimum Installs',  'Size',
    'Developer Website', 'Developer Email', 'Privacy Policy', 'App Name', 'Minimum Android', 'Content Rating', 
    'Editors Choice'], axis=1)

In [102]:
cat_name = 'Super Category'
df[cat_name] = df['Category']
df[cat_name] = df[cat_name].replace(['Adventure', 'Racing', 'Puzzle', 'Entertainment', 'Arcade', 'Photography', 'Sports', 'Card', 'Trivia', 'Strategy', 'Action', 'Simulation', 'Casino', 'Comics'], 'Entertainment')
df[cat_name] = df[cat_name].replace(['Communication', 'Social', 'Events', 'Dating', 'Role Playing', 'Lifestyle', 'Personalization', 'Medical', 'Health & Fitness', 'Beauty', 'Parenting'], 'Personal & Social')
df[cat_name] = df[cat_name].replace(['Productivity', 'Books & Reference', 'Education', 'Business', 'Educational', 'Finance'], 'Development')
df[cat_name] = df[cat_name].replace(['Casual', 'Tools', 'Libraries & Demo', 'Maps & Navigation', 'Travel & Local', 'Food & Drink', 'Music', 'Auto & Vehicles','Shopping', 'Board', 'Music & Audio', 'News & Magazines', 'Art & Design', 'House & Home', 'Weather', 'Word', 'Video Players & Editors'], 'Other')
df[cat_name] = df[cat_name].astype('category')
df[cat_name].value_counts()

Other                736774
Development          668061
Personal & Social    460998
Entertainment        447111
Name: Super Category, dtype: int64

In [103]:
df['Rating Bin'] = pd.cut(df['Rating'],bins=[0,3.9,4.4,5], labels=['Bad App Yo', 'Moderate', 'Superb'])
# labels=['Bad App Yo', 'Moderate', 'Superb']

In [104]:
#date_of_dataset = 2021
# df['Time since last update'] = pd.to_datetime(df['Last Updated']).apply(lambda date: date_of_dataset - int(date.year))

In [105]:
df.isnull().sum()

App Id                    0
Category                  0
Rating                22883
Rating Count          22883
Maximum Installs          0
Free                      0
Price                     0
Released              71053
Last Updated              0
Ad Supported              0
In App Purchases          0
Super Category            0
Rating Bin          1082645
dtype: int64

In [106]:
df = df.dropna()

In [116]:
df['Last Updated'] = pd.to_datetime(df['Last Updated'])
df['Released'] = pd.to_datetime(df['Released'])
df['Diff Last updated and Release'] = (df['Last Updated'] - df['Released']).dt.days

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1203357 entries, 1 to 2312943
Data columns (total 14 columns):
 #   Column                         Non-Null Count    Dtype         
---  ------                         --------------    -----         
 0   App Id                         1203357 non-null  object        
 1   Category                       1203357 non-null  object        
 2   Rating                         1203357 non-null  float64       
 3   Rating Count                   1203357 non-null  float64       
 4   Maximum Installs               1203357 non-null  int64         
 5   Free                           1203357 non-null  bool          
 6   Price                          1203357 non-null  float64       
 7   Released                       1203357 non-null  datetime64[ns]
 8   Last Updated                   1203357 non-null  datetime64[ns]
 9   Ad Supported                   1203357 non-null  bool          
 10  In App Purchases               1203357 non-null  bool 

In [119]:
# minder dan 100.000 installs en minstens 10 ratings
df = df[(df['Maximum Installs'] < 100000) & (df['Rating Count'] >= 10)]

In [120]:
df['Rating Bin'].value_counts()

Superb        276811
Bad App Yo    261710
Moderate      250407
Name: Rating Bin, dtype: int64

**Code below is still in development**

In [121]:
def prepare_inputs(encoder_type, feature):
    try:
        ohe = encoder_type
        return df.join(pd.DataFrame(data=ohe.fit_transform(df[[feature]]).toarray(), dtype=np.int8, 
            columns=list(df[feature].unique())))
    except:
        print("Get right column")

# reset index to be able to perform the join
df = df.reset_index(drop=True)

# one hot encoder
df = prepare_inputs(sklearn.preprocessing.OneHotEncoder(), 'Rating Bin')
df = prepare_inputs(sklearn.preprocessing.OneHotEncoder(), 'Super Category')

In [122]:
df.columns

Index(['App Id', 'Category', 'Rating', 'Rating Count', 'Maximum Installs',
       'Free', 'Price', 'Released', 'Last Updated', 'Ad Supported',
       'In App Purchases', 'Super Category', 'Rating Bin',
       'Diff Last updated and Release', 'Moderate', 'Superb', 'Bad App Yo',
       'Other', 'Personal & Social', 'Entertainment', 'Development'],
      dtype='object')

In [123]:
df = df.drop(['Last Updated', 'Rating', 'Category', 'Super Category', 'Rating Bin', 'Other', 'Released'], axis=1)
# 'Rating Bin'

In [124]:
df.head()

,App Id,Rating Count,Maximum Installs,Free,Price,Ad Supported,In App Purchases,Diff Last updated and Release,Moderate,Superb,Bad App Yo,Personal & Social,Entertainment,Development
0,com.webserveis.batteryinfo,64.0,7662,True,0.0,True,False,350,0,1,0,0,1,0
1,getfreedata.superfatiza.unlimitedjiodataprank,12.0,2567,True,0.0,True,False,4,0,0,1,0,1,0
2,com.mozaix.simoneboard,39.0,702,True,0.0,False,False,0,1,0,0,0,0,1
3,com.ikeyboard.theme.neon_3d.iron.tech,820.0,62433,True,0.0,True,False,381,0,0,1,0,0,1
4,com.MrScratchEnterprises.CarDogeGame,55.0,329,True,0.0,False,False,0,0,0,1,1,0,0


In [125]:
df['Rating Count'] = df['Rating Count'].astype('int')

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788928 entries, 0 to 788927
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   App Id                         788928 non-null  object 
 1   Rating Count                   788928 non-null  int32  
 2   Maximum Installs               788928 non-null  int64  
 3   Free                           788928 non-null  bool   
 4   Price                          788928 non-null  float64
 5   Ad Supported                   788928 non-null  bool   
 6   In App Purchases               788928 non-null  bool   
 7   Diff Last updated and Release  788928 non-null  int64  
 8   Moderate                       788928 non-null  int8   
 9   Superb                         788928 non-null  int8   
 10  Bad App Yo                     788928 non-null  int8   
 11  Personal & Social              788928 non-null  int8   
 12  Entertainment                 

In [127]:
df.to_parquet('data\Google-Playstore-Modified.parquet')